In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('data.csv')
data['month'] = data['month'].apply(lambda x: datetime.strptime(x, '%Y-%m')) 
data.set_index(['month'],inplace=True)

In [3]:
def generate_grids(n_grids):
    # shorting not allowed

    # Set a random seed for reproducibility
    np.random.seed(535)

    # Generate a DataFrame with 1000 rows and 4 columns
    rand_data1 = np.random.uniform(0, 1, size=(n_grids*10, 2))
    rand_data2 = np.random.uniform(-0.25, 0.75, size=(n_grids*10, 2))
    df_grid = pd.DataFrame(np.concatenate((rand_data1, rand_data2), axis=1), columns=['x1', 'x2', 'x3','x4'])

    # Calculate x4 as 1 minus the sum of x1 and x2
    df_grid['x5'] = 1 - df_grid['x1'] - df_grid['x2']

    # Drop rows based on the condition
    condition = (df_grid['x5'] >= -1) & (df_grid['x5'] <= 0) & (df_grid['x1']+df_grid['x2']+df_grid['x5']==1)
    df_grid = df_grid[condition]

    df_grid.reset_index(drop=True,inplace=True)
    df_grid = df_grid.iloc[:n_grids,:]

    # Display the sorted DataFrame
    return df_grid

In [4]:
from PathGen import *

def Experiments(MC,path_length,data,CAP,SAVE,g,ratio,year,tax=0.25):
    # MC simulation
    Simu_table = pd.DataFrame(index = range(MC),\
    columns = ['cap','sharpe','sortino','mdd'])
    year -= 1
    for mc in range(MC):
        mcdf = Path_Generator(path_length, data)
        temdf = MC_generate(mcdf,CAP,SAVE,g,ratio)
        temdf.reset_index(inplace=True,drop=True)
        # Loc out end of different years
        rst = temdf.loc[year,'cap_total']
        rst2 = temdf.loc[year,'cap_input'] 
        # Total Capital after tax for different dates
        Simu_table.iloc[mc,0] = rst - tax*(rst - rst2) # Tax Adjustment (only pay for capital gain)
        Simu_table.iloc[mc,1] = ret_annual_sharpe(temdf.iloc[:year,1])
        Simu_table.iloc[mc,2] = ret_annual_sortino(temdf.iloc[:year,1])
        Simu_table.iloc[mc,3] = max_drawdown2(temdf.iloc[:year,1])
        
    return Simu_table

In [5]:
def get_sim_stats(df_grid,n_grids,MC,path_length,data,CAP,SAVE,g,year,tax):
    # returns a df with each row corresponds to one MC sim, 
    # each col corresponds to one candidate objective function values
    df_compare = pd.DataFrame(index=range(n_grids),columns = ['Utility','Sharpe','Sortino','Mdd','Prob'])

    for idx in tqdm(range(n_grids)):
        ratio = (df_grid.iloc[idx,:]).to_list()
        temp_df = Experiments(MC,path_length,data,CAP,SAVE,g,ratio,year,tax)
        uiti = temp_df['cap'].apply(lambda x: np.log(x))
        df_compare.iloc[idx,0] = np.nanmean(uiti)
        df_compare.iloc[idx,1] = np.nanmean(temp_df['sharpe'])
        df_compare.iloc[idx,2] = np.nanmean(temp_df['sortino'])
        df_compare.iloc[idx,3] = np.nanmean(temp_df['mdd'])
        df_compare.iloc[idx,4] = (temp_df['cap'] >= 500000).mean()
        
    df_compare = df_compare.fillna(0)
    df_compare = df_compare[df_compare['Mdd']<=0.25] 
    # putting constraint on the max dd upper bound
    return df_compare

In [6]:
def get_sim_results(df_grid, df_compare):
    # the optimal allocation for each of the objectives
    df_results = pd.DataFrame(index=['uti','sharpe','sortino','mdd','prob'],\
                            columns=['x1','x2','x3','x4','x5'])

    df_results.iloc[0,:] = df_grid.loc[df_compare[['Utility']].idxmax(),:]
    df_results.iloc[1,:] = df_grid.loc[df_compare[['Sharpe']].idxmax(),:]
    df_results.iloc[2,:] = df_grid.loc[df_compare[['Sortino']].idxmax(),:]
    df_results.iloc[3,:] = df_grid.loc[df_compare[['Mdd']].idxmin(),:]
    df_results.iloc[4,:] = df_grid.loc[df_compare[['Prob']].idxmax(),:]

    # the corresponding objective values of the optimal allcoations above
    df_results_val = pd.DataFrame(index=['uti','sharpe','sortino','mdd','prob'],\
                            columns=['Utility','Sharpe','Sortino','Mdd','Prob'])

    df_results_val.iloc[0,:] = df_compare.loc[df_compare[['Utility']].idxmax(),:]
    df_results_val.iloc[1,:] = df_compare.loc[df_compare[['Sharpe']].idxmax(),:]
    df_results_val.iloc[2,:] = df_compare.loc[df_compare[['Sortino']].idxmax(),:]
    df_results_val.iloc[3,:] = df_compare.loc[df_compare[['Mdd']].idxmin(),:]
    df_results_val.iloc[4,:] = df_compare.loc[df_compare[['Prob']].idxmax(),:]
    
    return df_results, df_results_val

In [10]:
SAVE = 40000
CAP = 50000
g = 0.11
tax_rate = 0.25
path_length = 120
year = 120 # in months
MC = 200
n_grids = 1500

df_grid = generate_grids(n_grids)
df_compare = get_sim_stats(df_grid,n_grids,MC,path_length,data,CAP,SAVE,g,year,tax_rate)
df_results, df_results_val = get_sim_results(df_grid,df_compare)

100%|██████████| 1500/1500 [59:26<00:00,  2.38s/it]


In [12]:
df_results
# the index should be 10 Y

,x1,x2,x3,x4,x5
uti,0.5962732590649619,0.9122335765711901,0.4404862891718153,0.7159900339578056,-0.508506835636152
sharpe,0.5644501475362365,0.44682985343413717,0.32973558959023175,0.6867829453005588,-0.011280000970373627
sortino,0.5644501475362365,0.44682985343413717,0.32973558959023175,0.6867829453005588,-0.011280000970373627
mdd,0.2632361868837034,0.7748056046246182,0.2158668753877403,0.19517770834299852,-0.038041791508321654
prob,0.3486118254906069,0.8302611520488421,0.19303079035108062,0.4527699182105931,-0.17887297753944897


In [13]:
df_results_val

,Utility,Sharpe,Sortino,Mdd,Prob
uti,13.883258389134276,1.4891707035608688,2.08445544702809,0.24584490010893323,1.0
sharpe,13.67614984085251,1.699866691637333,2.3305779174230175,0.22909296873798354,1.0
sortino,13.67614984085251,1.699866691637333,2.3305779174230175,0.22909296873798354,1.0
mdd,13.42215257184696,1.113509026364522,1.6058522351375115,0.13494026790394284,0.98
prob,13.560737026725382,1.2170526375857842,1.748884876668657,0.15803461635177335,1.0


In [14]:
# import pickle

# with open('storage cache/df_compare.pkl', 'wb') as file:
#     pickle.dump(df_compare, file)
    
# with open('storage cache/df_results.pkl', 'wb') as file:
#     pickle.dump(df_results, file)

# with open('storage cache/df_results_val.pkl', 'wb') as file:
#     pickle.dump(df_results_val, file)